### Lendo os dados e importando as bibliotecas 

In [ ]:
from pandas import read_csv
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd

In [ ]:
df = read_csv('../Data/ReadyToUse2_Sintomas Musculoesqueléticos em Universitários - Respostas ao formulário 1.csv', sep = ",", encoding = 'latin1', engine = 'python', thousands = '.', decimal = ',')

In [ ]:
df.head(2)
df.columns

### Criando Distribuição de Frequência por IDADE

In [ ]:


# Criar a tabela de frequência absoluta
freq_table = df['IDADE'].value_counts().sort_index().reset_index()
freq_table.columns = ['IDADE', 'Frequency']

# Calcular a frequência relativa (porcentAGEM)
total_pessoas = freq_table['Frequency'].sum()
freq_table['PercentIDADE'] = (freq_table['Frequency'] / total_pessoas) * 100

# Exibir a tabela de frequência
print("Tabela de Frequência com Percentuais:")
print(freq_table)

# Gráfico de frequência absoluta com rótulos
plt.figure(figsize=(16, 10))
sns.barplot(x='IDADE', y='Frequency', data=freq_table, color='#D3185D')
plt.title('Distribuição de Frequência das Idades (Número de Pessoas)')
plt.xlabel('Idade')
plt.ylabel('Número de Pessoas')
plt.xticks(rotation=90)

# Adicionando rótulos de valor acima das colunas
for index, row in freq_table.iterrows():
    plt.text(x=index, y=row['Frequency'] + 0.5, s=int(row['Frequency']), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Gráfico de frequência relativa com rótulos de percentual
plt.figure(figsize=(16, 10))
sns.barplot(x='IDADE', y='PercentIDADE', data=freq_table, color='#D3185D')
plt.title('Distribuição de Frequência das Idades (%)')
plt.xlabel('Idade')
plt.ylabel('Percentual (%)')
plt.xticks(rotation=90)

# Adicionando rótulos de percentual acima das colunas
for index, row in freq_table.iterrows():
    plt.text(x=index, y=row['PercentIDADE'] + 0.5, s=f"{row['PercentIDADE']:.1f}%", ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


n = len(df['IDADE'])
k = math.ceil(np.log2(n) + 1)

# Criando os bins automaticamente
min_idade = min(df['IDADE'])
max_idade = max(df['IDADE'])
bins = np.linspace(min_idade, max_idade, k + 1)

# Labels dinâmicos para os bins
labels = [f'{int(bins[i])}-{int(bins[i+1]-1)}' for i in range(len(bins)-1)]

# Criando a tabela de frequência
faixas_etarias = pd.cut(df['IDADE'], bins=bins, labels=labels, include_lowest=True, right=False)
frequencia_absoluta = faixas_etarias.value_counts().sort_index()
frequencia_relativa = (frequencia_absoluta / n * 100).round(2)

# Montando a tabela
tabela_frequencia = pd.DataFrame({
    'Faixa Etária': labels,
    'Frequência': frequencia_absoluta.values,
    'Porcentagem (%)': frequencia_relativa.values
})

print(tabela_frequencia)

# Gráfico de barras com frequência absoluta
plt.figure(figsize=(8, 5))
frequencia_absoluta.plot(kind='bar', color='lightgreen', edgecolor='black')
plt.title('Distribuição de Faixa Etária na Amostra')
plt.xlabel('Faixa Etária')
plt.ylabel('Frequência Absoluta')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Criando Distribuição de Frequência por Curso, Sexo, Estado CIVIL, Raça e Renda

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Função para gerar tons de rosa
def generate_soft_pink_gradient(base_color, n_colors):
    base_rgb = tuple(int(base_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
    colors = []
    for i in range(n_colors):
        factor = (i / (n_colors - 1)) ** 0.7
        start_color = (240, 120, 160)
        interpolated = tuple(int(start_color[j] * (1-factor) + base_rgb[j] * factor) for j in range(3))
        colors.append('#%02x%02x%02x' % interpolated)
    return colors

cor_base = '#c81d5a'

variaveis = ['CURSO', 'SEXO', 'ESTADO CIVIL', 'RAÇA/COR DA PELE', 'RENDA FAMILIAR']

for var in variaveis:
    # Criar e ordenar tabela de frequência
    freq_table = df[var].value_counts(dropna=False).reset_index()
    freq_table.columns = [var, 'Frequência']
    freq_table = freq_table.sort_values('Frequência')  # Ordenar por frequência
    
    total = freq_table['Frequência'].sum()
    freq_table['Percentual (%)'] = (freq_table['Frequência'] / total) * 100

    print(f"\n📊 Tabela de Frequência - {var}")
    print(freq_table)

    # Paleta de cores
    n_categorias = len(freq_table)
    paleta = generate_soft_pink_gradient(cor_base, n_categorias) if n_categorias > 1 else [cor_base]

    # GRÁFICO DE FREQUÊNCIA ABSOLUTA
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(x=var, y='Frequência', data=freq_table, palette=paleta, order=freq_table[var])
    
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height + 0.02*total,
                f"{int(height)}",
                ha="center", va="bottom", fontsize=10)
    
    plt.title(f'Distribuição de {var} - Contagem', fontsize=14, pad=20)
    plt.ylabel('Número de Indivíduos', fontsize=12)
    plt.xlabel('')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(0, max(freq_table['Frequência']) * 1.18)
    plt.tight_layout()
    plt.show()

    # GRÁFICO DE FREQUÊNCIA RELATIVA
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(x=var, y='Percentual (%)', data=freq_table, palette=paleta, order=freq_table[var])
    
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height + 0.5,
                f"{height:.1f}%",
                ha="center", va="bottom", fontsize=10)
    
    plt.title(f'Distribuição de {var} - Proporção', fontsize=14, pad=20)
    plt.ylabel('Percentual (%)', fontsize=12)
    plt.xlabel('')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(0, max(freq_table['Percentual (%)']) * 1.18)
    plt.tight_layout()
    plt.show()

### Análise Descritiva sob dados demográficos na amostragem

In [ ]:


colunas_demograficas = df.columns[:6]

for i in colunas_demograficas:
    print(f"{i}")
    print(f"{df[f'{i}'].value_counts(normalize=True)*100} --- {df[f'{i}'].value_counts()}")

    print("\n\n\n\n")



### Análise descritiva sob dados sintomáticos e de bem estar na amostragem

In [ ]:

colunas_bem_estar_e_sintomas = df.columns[6:]

for i in colunas_bem_estar_e_sintomas:
    print(f"{i}")
    print(f"{df[f'{i}'].value_counts(normalize=True)*100} --- {df[f'{i}'].value_counts()}")

